La tabla "tbl_flujo_reporte" debe resumir en cada "id_item" el contenido del campo NETO de la tabla "tbl_transacciones" según "ITEM_COD", "ANIO" y "MES"

Evaluar las siguientes opciones:
1. ir creando una columna en tabla "tbl_flujo_reporte" para resumir la información de cada mes/año
2. Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD",   "ANIO" y "MES"

========================================================================================

1. **Crear una columna en la tabla "tbl_flujo_reporte" para resumir la información de cada mes/año**: 
* Esta opción implica agregar una nueva columna para cada mes/año en la tabla "tbl_flujo_reporte". Cada vez que se ingresen nuevos datos en la tabla "tbl_transacciones", tendrás que actualizar las columnas correspondientes en "tbl_flujo_reporte". 
* Esto podría ser eficiente si necesitas acceder rápidamente a los totales mensuales/año sin tener que realizar cálculos complejos en tiempo de consulta. 
* Sin embargo, podría volverse complicado si hay muchos meses/años diferentes o si la cantidad de datos es grande, ya que tendrías muchas columnas en la tabla "tbl_flujo_reporte".

2. **Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD", "ANIO" y "MES"**: 
* En esta opción, en lugar de tener una columna para cada mes/año en "tbl_flujo_reporte", tendrías una fila por cada combinación de "ITEM_COD", "ANIO" y "MES", con una columna que almacena el total NETO correspondiente. 
* Esto sería más escalable y flexible, ya que no tendrías que modificar la estructura de la tabla cada vez que agregues un nuevo mes/año. 
* Además, permitiría consultas más flexibles y eficientes sobre los datos agregados.

En general, la opción 2 parece ser la más flexible y escalable, ya que te permite almacenar los totales NETO de manera estructurada y realizar consultas eficientes sobre ellos. Sin embargo, la elección final dependerá de tus necesidades específicas y de la complejidad de tu aplicación.

========================================================================================

Uno de los propósitos de la tabla "tbl_flujo_reporte" es que sirva como puente para tomar datos que alimentarán un reporte

* Si el propósito principal de la tabla "tbl_flujo_reporte" es alimentar un reporte, entonces la opción 2 parece ser la más adecuada. 
* Al tener una fila por cada combinación de "ITEM_COD", "ANIO" y "MES" con el total NETO correspondiente, puedes generar fácilmente los datos necesarios para el reporte utilizando consultas SQL simples.
* Además, al utilizar esta estructura, la tabla "tbl_flujo_reporte" actuará como un puente eficiente para obtener los datos agregados necesarios para el reporte, ya que los totales NETO estarán precalculados y disponibles en la tabla, lo que evitará la necesidad de realizar cálculos complejos en tiempo de consulta.

Por lo tanto, la opción 2 te proporcionará una estructura eficiente y escalable para alimentar el reporte, lo que facilitará el proceso de generación del mismo y garantizará un rendimiento óptimo, especialmente si estás trabajando con grandes volúmenes de datos.

========================================================================================

In [30]:
import pandas as pd

import mysql.connector
import configparser
from openpyxl import Workbook

import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

In [31]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# ============================================================================================================

def report_mes(anio_inicio, mes_inicio, anio_fin, mes_fin):

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_flujo_reporte
    query = f"""
        SELECT *
        FROM tbl_flujo_reporte
        WHERE (anio > {anio_inicio} OR (anio = {anio_inicio} AND mes >= {mes_inicio}))
        AND (anio < {anio_fin} OR (anio = {anio_fin} AND mes <= {mes_fin}))
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    reporte_mes = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return reporte_mes

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
reporte_mes = report_mes(2023, 1, 2023, 2)
# print(reporte_mes)

# Crea la columna 'periodo' concatenando las columnas 'anio', 'mes'
reporte_mes['periodo'] = pd.to_datetime(reporte_mes['anio'].astype(str) + '-' + reporte_mes['mes'].astype(str) )
reporte_mes.tail()

C:\Users\gusta\AppData\Local\Temp\ipykernel_1044\3356292400.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reporte_mes = pd.read_sql(query, conexion)


,id_item,id_item_cod,anio,mes,total_neto,periodo
33,1791,32,2023,2,-510629.00,2023-02-01
34,1792,35,2023,2,-1264000.00,2023-02-01
35,1793,38,2023,2,-776881.77,2023-02-01
36,1794,39,2023,2,-98000.00,2023-02-01
37,1795,40,2023,2,-292500.00,2023-02-01


In [32]:
reporte_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_item      38 non-null     int64         
 1   id_item_cod  38 non-null     int64         
 2   anio         38 non-null     int64         
 3   mes          38 non-null     int64         
 4   total_neto   38 non-null     float64       
 5   periodo      38 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 1.9 KB


In [33]:
# ============================================================================================================
# Crea un dataframe resumen de reporte_mes con la suma de 'total_neto' por 'id_item_cod' y 'periodo'
# ============================================================================================================

# Crear el DataFrame reporte_final utilizando pivot_table
reporte_final = pd.pivot_table(reporte_mes, 
                               values='total_neto', 
                               index='id_item_cod', 
                               columns='periodo', 
                               aggfunc='sum')

# Conservar el formato datetime en los títulos de las columnas
reporte_final.columns = reporte_final.columns.strftime('%Y-%m')


reporte_final.reset_index(inplace = True)

# Mostrando el nuevo dataframe
reporte_final

periodo,id_item_cod,2023-01,2023-02
0,2,-330000.00,-330000.00
1,3,-393648.86,-456206.92
2,5,NaN,-37000.00
3,7,-2220953.00,-3558973.08
4,8,-2350098.00,-4631322.00
5,9,39685569.00,45085363.00
6,11,-3338960.00,-3826570.00
7,12,-115973.92,-112080.15
8,14,2898.16,2323.64
9,15,-5632376.00,-5281700.00


In [34]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  22 non-null     int64  
 1   2023-01      18 non-null     float64
 2   2023-02      20 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 656.0 bytes


In [35]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_saldo_mes según rango definido en parámetros de entrada
# ============================================================================================================

def saldo_mes(anio_inicio, mes_inicio, anio_fin, mes_fin):

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_saldo_mes
    query = f"""
        SELECT *
        FROM tbl_saldo_mes
        WHERE (anio > {anio_inicio} OR (anio = {anio_inicio} AND mes >= {mes_inicio}))
        AND (anio < {anio_fin} OR (anio = {anio_fin} AND mes <= {mes_fin}))
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    saldos_mes = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return saldos_mes

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_saldo_mes según rango definido en parámetros de entrada

# Ejemplo de uso:
inf_saldos = saldo_mes(2023, 1, 2023, 2)
# print(reporte_mes)

# Crea la columna 'periodo' concatenando las columnas 'anio', 'mes'
inf_saldos['periodo'] = pd.to_datetime(inf_saldos['anio'].astype(str) + '-' + inf_saldos['mes'].astype(str) )
inf_saldos.tail()

C:\Users\gusta\AppData\Local\Temp\ipykernel_1044\268230371.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saldos_mes = pd.read_sql(query, conexion)


,id_saldo_mes,fecha,mes,anio,saldo_inicial,neto_mes,saldo_final,periodo
0,30,2023-01-31,1,2023,39647234.51,-15151744.01,24495490.50,2023-01-01
1,31,2023-02-28,2,2023,24495490.50,-17049613.28,7445877.22,2023-02-01


In [36]:
inf_saldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_saldo_mes   2 non-null      int64         
 1   fecha          2 non-null      object        
 2   mes            2 non-null      int64         
 3   anio           2 non-null      int64         
 4   saldo_inicial  2 non-null      float64       
 5   neto_mes       2 non-null      float64       
 6   saldo_final    2 non-null      float64       
 7   periodo        2 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 256.0+ bytes


In [37]:
# ============================================================================================================
# Crea un dataframe resumen de inf_saldos
# con la suma de 'total_neto' por 'id_item_cod' y 'periodo'
# ============================================================================================================

# Crear el DataFrame reporte_final utilizando pivot_table
saldos_final = pd.pivot_table(inf_saldos, 
                               values='saldo_inicial', 
                               columns='periodo', 
                               aggfunc='sum')

# Conservar el formato datetime en los títulos de las columnas
saldos_final.columns = saldos_final.columns.strftime('%Y-%m')

saldos_final.reset_index(inplace = True)

# Mostrando el nuevo dataframe
saldos_final

periodo,index,2023-01,2023-02
0,saldo_inicial,39647234.51,24495490.5


In [38]:
saldos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    1 non-null      object 
 1   2023-01  1 non-null      float64
 2   2023-02  1 non-null      float64
dtypes: float64(2), object(1)
memory usage: 152.0+ bytes


In [39]:
# ============================================================================================================
# Renombrar la columna 'index' de la tabla saldos_final a 'id_item_cod'
# ============================================================================================================
saldos_final.rename(columns={'index':'id_item_cod'}, inplace=True)
saldos_final

periodo,id_item_cod,2023-01,2023-02
0,saldo_inicial,39647234.51,24495490.5


In [40]:
saldos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  1 non-null      object 
 1   2023-01      1 non-null      float64
 2   2023-02      1 non-null      float64
dtypes: float64(2), object(1)
memory usage: 152.0+ bytes


In [41]:
# ============================================================================================================
# Cambiar el contenido del registro en la columna 'id_item_cod' de la tabla saldos_final
# ============================================================================================================
saldos_final['id_item_cod'] = saldos_final['id_item_cod'].replace('saldo_inicial', '48')
saldos_final['id_item_cod'] = saldos_final['id_item_cod'].astype(int)

saldos_final

periodo,id_item_cod,2023-01,2023-02
0,48,39647234.51,24495490.5


In [42]:
saldos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  1 non-null      int32  
 1   2023-01      1 non-null      float64
 2   2023-02      1 non-null      float64
dtypes: float64(2), int32(1)
memory usage: 148.0 bytes


In [43]:
print(type(reporte_final))

<class 'pandas.core.frame.DataFrame'>


In [44]:
# Concatenar los DataFrames
reporte_final = pd.concat([reporte_final, saldos_final], ignore_index=True)
reporte_final

periodo,id_item_cod,2023-01,2023-02
0,2,-330000.00,-330000.00
1,3,-393648.86,-456206.92
2,5,NaN,-37000.00
3,7,-2220953.00,-3558973.08
4,8,-2350098.00,-4631322.00
5,9,39685569.00,45085363.00
6,11,-3338960.00,-3826570.00
7,12,-115973.92,-112080.15
8,14,2898.16,2323.64
9,15,-5632376.00,-5281700.00


In [45]:
# ============================================================================================================
# En este punto se tiene un DataFrame con la información de los saldos iniciales y los registros resumidos de flujos de los meses
# ============================================================================================================

In [46]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_fcdir_items
# ============================================================================================================

def data_items():

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_fcdir_items
    query = f"""
        SELECT *
        FROM tbl_fcdir_items
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    tabla_items = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return tabla_items

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
tabla_items = data_items()
tabla_items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_item  45 non-null     int64 
 1   name     45 non-null     object
 2   id_grp   45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


C:\Users\gusta\AppData\Local\Temp\ipykernel_1044\4204913904.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabla_items = pd.read_sql(query, conexion)


In [47]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_fcdir_grupos
# ============================================================================================================

def data_grupos():

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_fcdir_grupos
    query = f"""
        SELECT *
        FROM tbl_fcdir_grupos
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    tabla_grupos = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return tabla_grupos

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
tabla_grupos = data_grupos()
tabla_grupos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id_grp  6 non-null      int64 
 1   name    6 non-null      object
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


C:\Users\gusta\AppData\Local\Temp\ipykernel_1044\812418555.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabla_grupos = pd.read_sql(query, conexion)


In [48]:
# ============================================================================================================
# Actualiza 'reporte_final' con datos de 'tabla_items'
# ============================================================================================================

# Realizar la fusión basada en la columna 'id_item_cod' de la tabla 'reporte_mes' y 'id_item' de la tabla 'tabla_items'
reporte_final = pd.merge(reporte_final, tabla_items, left_on='id_item_cod', right_on='id_item', how='left')

# Eliminar la columna 'id_item' ya que se ha utilizado para hacer la fusión y ahora está duplicada
# reporte_final.drop('id_item_y', axis=1, inplace=True)

# Mostrar el DataFrame resultante con la información agregada
reporte_final.head()


,id_item_cod,2023-01,2023-02,id_item,name,id_grp
0,2,-330000.00,-330000.00,2,Accounting software,4
1,3,-393648.86,-456206.92,3,Bank comissions & related,4
2,5,NaN,-37000.00,5,Certificates,4
3,7,-2220953.00,-3558973.08,7,Cleaning service,4
4,8,-2350098.00,-4631322.00,8,Cleaning supplies,4


In [49]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  23 non-null     int64  
 1   2023-01      19 non-null     float64
 2   2023-02      21 non-null     float64
 3   id_item      23 non-null     int64  
 4   name         23 non-null     object 
 5   id_grp       23 non-null     int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 1.2+ KB


In [50]:
# ============================================================================================================
# Actualiza 'reporte_final' con datos de 'tabla_grupos'
# ============================================================================================================

# Realizar la fusión basada en la columna 'id_grp' de la tabla 'reporte_final' con la columna 'id_grp' de la tabla 'tabla_grupos'
reporte_final = pd.merge(reporte_final, tabla_grupos, left_on='id_grp', right_on='id_grp', how='left')
reporte_final

,id_item_cod,2023-01,2023-02,id_item,name_x,id_grp,name_y
0,2,-330000.00,-330000.00,2,Accounting software,4,OPERATING
1,3,-393648.86,-456206.92,3,Bank comissions & related,4,OPERATING
2,5,NaN,-37000.00,5,Certificates,4,OPERATING
3,7,-2220953.00,-3558973.08,7,Cleaning service,4,OPERATING
4,8,-2350098.00,-4631322.00,8,Cleaning supplies,4,OPERATING
5,9,39685569.00,45085363.00,9,Collect from clients,4,OPERATING
6,11,-3338960.00,-3826570.00,11,Facility: Electricity,4,OPERATING
7,12,-115973.92,-112080.15,12,Hosting & domains,4,OPERATING
8,14,2898.16,2323.64,14,Interests received,2,FINANCING
9,15,-5632376.00,-5281700.00,15,Internet,4,OPERATING


In [51]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  23 non-null     int64  
 1   2023-01      19 non-null     float64
 2   2023-02      21 non-null     float64
 3   id_item      23 non-null     int64  
 4   name_x       23 non-null     object 
 5   id_grp       23 non-null     int64  
 6   name_y       23 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.4+ KB


In [52]:
# ============================================================================================================
# Depura columnas de 'reporte_final'
# ============================================================================================================
reporte_final.drop(columns=['id_item'], inplace=True)
reporte_final.rename(columns={'name_x': 'Concepto'}, inplace=True)
reporte_final.rename(columns={'name_y': 'Grupo'}, inplace=True)

reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  23 non-null     int64  
 1   2023-01      19 non-null     float64
 2   2023-02      21 non-null     float64
 3   Concepto     23 non-null     object 
 4   id_grp       23 non-null     int64  
 5   Grupo        23 non-null     object 
dtypes: float64(2), int64(2), object(2)
memory usage: 1.2+ KB


In [53]:
# ============================================================================================================
# Cambia el orden de columnas en 'reporte_final'
# ============================================================================================================

# Extraer las columnas 'Concepto', 'id_grp' y 'Grupo'
concepto_col = reporte_final.pop('Concepto')
id_grp_col = reporte_final.pop('id_grp')
grupo_col = reporte_final.pop('Grupo')

# Insertar las columnas en la nueva ubicación
reporte_final.insert(1, 'Concepto', concepto_col)
reporte_final.insert(2, 'id_grp', id_grp_col)
reporte_final.insert(3, 'Grupo', grupo_col)

reporte_final


,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
0,2,Accounting software,4,OPERATING,-330000.00,-330000.00
1,3,Bank comissions & related,4,OPERATING,-393648.86,-456206.92
2,5,Certificates,4,OPERATING,NaN,-37000.00
3,7,Cleaning service,4,OPERATING,-2220953.00,-3558973.08
4,8,Cleaning supplies,4,OPERATING,-2350098.00,-4631322.00
5,9,Collect from clients,4,OPERATING,39685569.00,45085363.00
6,11,Facility: Electricity,4,OPERATING,-3338960.00,-3826570.00
7,12,Hosting & domains,4,OPERATING,-115973.92,-112080.15
8,14,Interests received,2,FINANCING,2898.16,2323.64
9,15,Internet,4,OPERATING,-5632376.00,-5281700.00


In [54]:
# ============================================================================================================
# Reemplaza valores NaN en 'reporte_final'
# ============================================================================================================

# Reemplazar los valores NaN en 'Concepto' por 0
reporte_final.fillna(0, inplace=True)
reporte_final

,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
0,2,Accounting software,4,OPERATING,-330000.00,-330000.00
1,3,Bank comissions & related,4,OPERATING,-393648.86,-456206.92
2,5,Certificates,4,OPERATING,0.00,-37000.00
3,7,Cleaning service,4,OPERATING,-2220953.00,-3558973.08
4,8,Cleaning supplies,4,OPERATING,-2350098.00,-4631322.00
5,9,Collect from clients,4,OPERATING,39685569.00,45085363.00
6,11,Facility: Electricity,4,OPERATING,-3338960.00,-3826570.00
7,12,Hosting & domains,4,OPERATING,-115973.92,-112080.15
8,14,Interests received,2,FINANCING,2898.16,2323.64
9,15,Internet,4,OPERATING,-5632376.00,-5281700.00


In [55]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  23 non-null     int64  
 1   Concepto     23 non-null     object 
 2   id_grp       23 non-null     int64  
 3   Grupo        23 non-null     object 
 4   2023-01      23 non-null     float64
 5   2023-02      23 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.2+ KB


In [56]:
# Order the dataframe by "Grupo"
reporte_final_sorted = reporte_final.sort_values(by="Grupo")
reporte_final_sorted

,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
8,14,Interests received,2,FINANCING,2898.16,2323.64
22,48,Saldo Iinicial,5,INITIAL BALANCE,39647234.51,24495490.50
20,39,ICA. Tax Payable,4,OPERATING,0.00,-98000.00
19,38,Video surveillance,4,OPERATING,-164126.98,-776881.77
18,35,Tax report: Witholding taxes,4,OPERATING,-1163000.00,-1264000.00
17,33,Tax report: ICA,4,OPERATING,-2712000.00,0.00
16,32,Stationery,4,OPERATING,-510629.00,-510629.00
15,30,Social security,4,OPERATING,0.00,-2862097.00
14,29,Rent - Offices,4,OPERATING,-28749303.00,-28749303.00
13,24,Payroll (Basics),4,OPERATING,-7105297.00,-9297023.00


In [65]:
# Crea el dataframe de ejemplo
df = pd.DataFrame({'A': ['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                   'B': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                   'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                   'D': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]})


# Función para insertar filas con subtotal y calcular subtotales
    # Obtener las columnas numéricas
    # Obtener el nombre de la columna de índice
    # Obtener Grupo actual: inicialmente el primer valor de la primera columna
    # Diccionario para almacenar subtotales del grupo actual
    # Lista para almacenar filas a concatenar
    # Iterar sobre las filas del DataFrame para calcular subtotales y agregar filas de subtotal
        # Compara el valor actual de la columna de agrupación con la fila anterior
            
            # En caso de que el valor de la columna de agrupación sea "diferente" al grupo actual:
            # Crea una fila con el nombre del grupo
            # Añade los subtotales al diccionario
            # Añade la fila de subtotal a la lista de filas a concatenar
            # Reinicia los subtotales para el nuevo grupo
            # Actualiza el grupo actual con el valor de la fila actual
        
        # En caso de que el valor de la columna de agrupación sea "igual" al grupo actual:
        # Actualiza los subtotales del grupo actual
        # Agrega la fila actual al final de las filas a concatenar

    # Agregar los subtotales del último grupo y concatenar las filas de subtotal al DataFrame original


# ===============================================================================================================

    # Aplica la función para insertar filas con subtotal y calcular subtotales
    # Imprime el dataframe con las filas insertadas, subtotales y subtotales calculados
    
    # Concatena las filas de subtotal al DataFrame original 

# Función para insertar filas con subtotal y calcular subtotales
def insertar_fila_subtotal(df):
    """
    Inserta una fila en blanco con "Subtotal" y calcula los subtotales de cada grupo en las columnas numéricas.
    Argumentos: df (pandas.DataFrame): El dataframe a procesar.
    Devuelve: pandas.DataFrame: El dataframe con las filas insertadas, subtotales y subtotales calculados.
    """
    # Obtener las columnas numéricas
    columnas_numericas = df.select_dtypes(include='number').columns
    print('Columnas numéricas: ', columnas_numericas)
    print("="*50)
    
    # Obtener el nombre de la columna de índice
    indice_columna = df.columns.tolist().index(columnas_numericas[0])
    print('Indice de columna: ',indice_columna)
    print("="*50)
    
    # Obtiene el valor de grupo actual: inicialmente el primer valor de la primera columna
    grupo_actual = df.iloc[0, indice_columna - 1]  
    print('Grupo actual: ',grupo_actual)
    print("="*50)

    # Inicializa diccionario con subtotales del grupo actual en 0 para cada columna numérica 
    subtotales_grupo = {columna: 0 for columna in columnas_numericas}  
    print('Subtotales del grupo: ',subtotales_grupo)
    print("="*50)
    
    # Inicializa lista para almacenar filas a concatenar
    rows_to_concat = []  
    print('Filas a concatenar: ',rows_to_concat)
    print("="*50)
    
    for index, fila in df.iterrows():   # Iterar sobre las filas del DataFrame
        print('valor de la fila anterior: ',fila.iloc[indice_columna - 1])
        print('Grupo actual: ',grupo_actual)
        print("-"*50)

        # En caso de que el valor de la columna de agrupación sea "diferente" al grupo actual
        if fila.iloc[indice_columna - 1] != grupo_actual: 
            subtotal = {'A': f"Subtotal {grupo_actual}"}                                            # Crea una fila con el nombre del grupo
            print('Subtotal: ',subtotal)
            print("-"*50)
            subtotal.update({columna: subtotales_grupo[columna] for columna in columnas_numericas}) # Añade los subtotales del grupo al diccionario
            print('Subtotal actualizado: ',subtotales_grupo)
            print("-"*50)
            rows_to_concat.append(subtotal)                                                         # Añade la fila de subtotal a la lista de filas a concatenar
            print('Filas a concatenar: ',rows_to_concat)
            print("-"*50)
            subtotales_grupo = {columna: 0 for columna in columnas_numericas}                       # Reinicia los subtotales para el nuevo grupo
            print("="*50)
            print('Reinicio de Subtotales del grupo',subtotales_grupo)
            print("-"*50)
            grupo_actual = fila.iloc[indice_columna - 1]                                            # Actualiza el grupo actual con el valor de la fila actual
            print('Grupo actualizado: ',grupo_actual)
            print("-"*50)

        # Sin importar si el grupo es igual o diferente, itera sobre las columnas numéricas para actualizar los subtotales                
        for columna in columnas_numericas:  
            subtotales_grupo[columna] += fila[columna]      # Suma el valor de la fila actual al subtotal del grupo actual
            print('itera para actualizar los subtotales')
            print('Subtotales del grupo actual: ',subtotales_grupo)
            print("-"*50)

        # Sin importar si el grupo es igual o diferente, siempre agrega la fila actual al final de la lista de filas a concatenar
        rows_to_concat.append(fila)
        print('Filas a concatenar: ',rows_to_concat)
        print("="*50)
    
    print("="*50)
    print(df)
    print("="*50)

    # Agregar los subtotales del último grupo
    subtotal = {'A': f"Subtotal {grupo_actual}"}
    subtotal.update({columna: subtotales_grupo[columna] for columna in columnas_numericas})
    rows_to_concat.append(subtotal)
    
    # Concatena las filas de subtotal al DataFrame original
    df = pd.concat([pd.DataFrame([row]) for row in rows_to_concat], ignore_index=True) # 
    
    return df

# Aplica la función para insertar filas con subtotal y calcular subtotales
df = insertar_fila_subtotal(df.copy())

# Imprime el dataframe con las filas insertadas, subtotales y subtotales calculados
print(df)


Columnas numéricas:  Index(['B', 'C', 'D'], dtype='object')
Indice de columna:  1
Grupo actual:  a
Subtotales del grupo:  {'B': 0, 'C': 0, 'D': 0}
Filas a concatenar:  []
valor de la fila anterior:  a
Grupo actual:  a
--------------------------------------------------
itera para actualizar los subtotales
Subtotales del grupo actual:  {'B': 1, 'C': 0, 'D': 0}
--------------------------------------------------
itera para actualizar los subtotales
Subtotales del grupo actual:  {'B': 1, 'C': 1, 'D': 0}
--------------------------------------------------
itera para actualizar los subtotales
Subtotales del grupo actual:  {'B': 1, 'C': 1, 'D': 2}
--------------------------------------------------
Filas a concatenar:  [A    a
B    1
C    1
D    2
Name: 0, dtype: object]
valor de la fila anterior:  a
Grupo actual:  a
--------------------------------------------------
itera para actualizar los subtotales
Subtotales del grupo actual:  {'B': 3, 'C': 1, 'D': 2}
--------------------------------------

In [58]:
# ============================================================================================================
# Exportar el DataFrame reporte_final a un archivo Excel
# ============================================================================================================

Hacer un reporte en excel:

Encabezado de dos filas (sin datos):
En la primera fila, el nombre de la empresa: "Empresa SHINA"
Segunda fila vacía

En las siguientes filas se insertarán los datos:
Tercera fila: información de saldo inicial: registro con id_grp = 5
Cuarta fila vacía
A partir de la quinta fila los registros con id_grp =  4 ordenados por valor ascendente de la columna del primer inicial
Siguiente fila que calcule subtotal de registros con id_grp =  4
Siguiente fila vacia
Siguiente grupo de filas,  los registros con id_grp =  3 ordenados por valor ascendente de la columna del primer inicial
Siguiente fila que calcule subtotal de registros con id_grp =  3
Siguiente fila vacia
Siguiente grupo de filas,  los registros con id_grp =  2 ordenados por valor ascendente de la columna del primer inicial
Siguiente fila que calcule subtotal de registros con id_grp =  2
Siguiente fila vacia
Siguiente fila que calcule el saldo final como la suma de registro con id_grp = 5 + subtotal de registros con id_grp =  2 + subtotal  de registros con id_grp = 3 + subtotal  de registros con id_grp =  4